# Kaggle 5-Day Gen AI Capstone

The objective is to use the Gemini API to demonstrate at least three (3) Gen AI capabilities. I plan to do most of them.

1. [x] Structured output/JSON mode/controlled generation
1. [x] Few-shot prompting
1. [x] Understanding a document
   - Text
   - Image
   - ~~Video~~
   - ~~Audio~~
1. [x] Function Calling
1. [x] Agents
1. ~~Long context window~~  Can process large documents
1. ~~Context caching~~  Cache content for repetitive use, e.g. video analyses
1. [ ] Gen AI evaluation
1. ~~Grounding~~  Search results complement responses to maintain freshness.
1. ~~Training~~  (Did similar at Aggregate Knowledge before BSchool)
   - Embeddings  Embed data into a matrix, reduce dimensions to look it up.   
   - Retrieval augmented generation (RAG)
   - Vector search/vector store/vector database
1. ~~MLOps (with GenAI)~~


In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

### Pull your API key

This notebook was written using Anaconda, rather than Kaggle secrets. Kaggle secrets code is commented out.

In [3]:
# from kaggle_secrets import UserSecretsClient
# 
# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
# 
# client = genai.Client(api_key=GOOGLE_API_KEY)

import os

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY", "default_value")

client = genai.Client(api_key=GOOGLE_API_KEY)

### Automated retry

The this project is deep and sends a lot of requests, so set up an automatic retry
that ensures your requests are retried when per-minute quota is reached.

In [4]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## 1. Understanding text and image documents

This section uses numerous models to evaluate a document. Source document was downloaded manually because it was prevented by the source server.

Capstone objectives covered:
* Understanding a text file
* Understanding an image file

In [5]:
# !wget -nv -O cpi.pdf https://www.bls.gov/news.release/pdf/cpi.pdf

document_file = client.files.upload(file='cpi.pdf')

### Summarizing a text document

The summarisation request used here is fairly basic. It targets the training content specifically but provides no guidance otherwise.

In [6]:
request = 'Tell me about the Consumer Price Index in July.'

def summarize_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarize_doc(request)
Markdown(summary)

Here's a summary of the Consumer Price Index (CPI) for July 2025, based on the provided news release:

*   **Overall CPI:** The CPI for All Urban Consumers (CPI-U) increased by 0.2 percent in July on a seasonally adjusted basis, following a 0.3 percent increase in June. Over the last 12 months, the all items index increased 2.7 percent before seasonal adjustment.
*   **Shelter:** The index for shelter rose 0.2 percent in July and was the primary factor in the all items monthly increase.
*   **Food:** The food index was unchanged over the month as the food away from home index rose 0.3 percent while the food at home index fell 0.1 percent.
*   **Energy:** The index for energy fell 1.1 percent in July as the index for gasoline decreased 2.2 percent over the month.
*   **All Items Less Food and Energy:** The index for all items less food and energy rose 0.3 percent in July, following a 0.2-percent increase in June.
*   **Other Notable Changes:** Indexes that increased over the month include medical care, airline fares, recreation, household furnishings and operations, and used cars and trucks. The indexes for lodging away from home and communication were among the few major indexes that decreased in July.
*   **12-Month Changes:** The all items index rose 2.7 percent for the 12 months ending July. The all items less food and energy index rose 3.1 percent over the last 12 months. The energy index decreased 1.6 percent for the 12 months ending July. The food index increased 2.9 percent over the last year.
*   **Wireless Telephone Services:** BLS has replaced survey data collected for the CPI's wireless telephone services index with secondary source data and non-traditional index methods.

In [7]:
request2 = 'Tell me about how the data is seasonally adjusted.'

def summarize_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.2)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary2 = summarize_doc(request2)
Markdown(summary2)

Certainly! Here's information about how the data is seasonally adjusted, based on the provided document:

**How Seasonal Adjustment is Done**

*   **Method:** The Consumer Price Index (CPI) program uses the X-13ARIMA-SEATS seasonal adjustment method to compute seasonally adjusted data.
*   **Seasonal Factors:** Seasonal factors are derived using this method.
*   **Annual Updates:** These factors are updated each February. The new factors are used to revise the previous 5 years of seasonally adjusted data.
*   **Availability of Factors:** The seasonal factors are available at www.bls.gov/cpi/tables/seasonal-adjustment/seasonal-factors-2025.xlsx.
*   **Intervention Analysis:** For some CPI series, the Bureau of Labor Statistics (BLS) uses intervention analysis seasonal adjustment (IASA). This process estimates and removes distortions caused by unusual events before calculating seasonal factors.

**Why Seasonal Adjustment is Used**

*   **Analyzing Short-Term Trends:** Seasonally adjusted changes are preferred for analyzing short-term price trends in the economy.
*   **Eliminating Typical Changes:** Seasonal adjustment eliminates the effect of changes that normally occur at the same time and in about the same magnitude every year (e.g., weather events, production cycles, model changeovers, holidays, and sales).
*   **Focusing on Atypical Changes:** This allows data users to focus on changes that are not typical for the time of year.

**Important Considerations**

*   **Unadjusted Data:** The unadjusted data are of primary interest to consumers concerned about the prices they actually pay.
*   **Escalation Agreements:** BLS advises against using seasonally adjusted data in escalation agreements because seasonally adjusted series are revised annually for five years.
*   **Revision of Indexes:** Seasonally adjusted data, including the U.S. city average all items index levels, are subject to revision for up to 5 years after their original release.
*   **Determining Seasonal Status:** Each year, the seasonal status of every series is reevaluated based upon certain statistical criteria.

I hope this is helpful!


### Define and employ an evaluation rubric

Instructed the LLM to evaluate the response using a clear definition and [assessment rubric](https://en.wikipedia.org/wiki/Rubric_%28academic%29).

#### Rubric
1. How well the model followed the prompt ("instruction following")
1. Whether it included relevant data in the prompt ("groundedness")
1. How easy the text is to read ("fluency"), or other factors like "verbosity" or "quality".

#### Prompt
- Defines an evaluation agent using a pre-written "summarization" prompt
- Uses it to gauge the quality of the previously-generated summary.

In [8]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, ai_response):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=[request, document_file], ai_response=summary)
Markdown(text_eval)

STEP 1: The AI-generated response followed the instruction of the prompt. The response is grounded in the document provided. The response is well-organized and easy to read. The response did a good job extracting all the key information from the document provided.
STEP 2: I would rate this response as very good.
RATING: 5


In this example, the model generated a textual justification that was set up in a chat context. This full text response is useful both for human interpretation and for giving the model a place to "collect notes" while it assesses the text and produces a final score. This "note taking" or "thinking" strategy typically works well with auto-regressive models, where the generated text is passed back into the model at each generation step. This means the working "notes" are used when generating final result output.

In the next turn, the model converts the text output into a structured response. If you want to aggregate scores or use them programatically then you want to avoid parsing the unstructured text output. Here the `SummaryRating` schema is passed, so the model converts the chat history into an instance of the `SummaryRating` enum.

In [9]:
struct_eval

<SummaryRating.VERY_GOOD: '5'>

### Refine the summary prompt

In [10]:
new_prompt = "Explain like I'm 5 the Consumer Price Index in July."

def run_and_eval_summary(a_prompt):
  """Generate and evaluate the summary using the new prompt."""
  summary = summarize_doc(a_prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_summary([a_prompt, document_file], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_summary(new_prompt)

Okay, imagine you have a big basket filled with all the things your family buys in a month: food, toys, clothes, gas for the car, and even going to the movies.

The Consumer Price Index (CPI) is like a number that tells us if the price of that whole basket went up or down compared to last month.

In July, the CPI went up by 0.2%. That means, on average, the things in your basket cost a little bit more than they did in June.

Some things in the basket went up more than others. For example, the price of shelter (like renting an apartment) went up. But other things, like energy (like gasoline), went down.

So, the CPI is just a way to see if things are getting more expensive or cheaper overall. In July, things got a little more expensive.
-----

## Evaluation
STEP 1: The AI response successfully simplifies and explains the Consumer Price Index (CPI) in a way that a 5-year-old could understand. It uses the analogy of a "basket" of goods to represent the CPI and explains how the index reflects whether the cost of these goods has increased or decreased compared to the previous month. It follows instructions, is grounded, is concise, and is fluent.

STEP 2: Based on the rubric, the response is very good.

## Rating
5

-----

SummaryRating.VERY_GOOD


### Understanding an image

Take an image and test AI comprehension.

In [ ]:
import PIL.Image

# Load the image
img_filename = 'IMG_1125.jpg'
img = PIL.Image.open(img_filename)

# Assign the prompt
img_prompt = "Describe this image:"
# img_prompt = "What type of tree is in the background of this image:"

def summarize_img(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.2)
  response = client.models.generate_content(
      model='gemini-1.5-pro-latest',
      config=config,
      contents=[request, img],
  )

  return response.text

def eval_img_summary(prompt, ai_response):
  """Evaluate the generated image summary against the prompt used."""

  chat = client.chats.create(model='gemini-1.5-pro-latest')

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


def run_and_eval_img_summary(prompt: str):
  """Generate and evaluate the image summary using the new prompt."""
  summary = summarize_img(prompt)
  display(Markdown(summary + '\n-----'))

  text, struct = eval_img_summary(
    
    
    
    
    -
    prompt, img], summary)
  display(Markdown(text + '\n-----'))
  print(struct)

run_and_eval_img_summary(img_prompt)
# summarize_img(img_prompt)

This is a medium-shot, slightly low-angle view of a section of a roof and the top of a utility pole against a backdrop of trees and a clear blue sky. 


The roof is covered in light brown composite shingles, with visible horizontal lines indicating the layered structure. Two vents are present on the roof. One is a silver, turbine-style vent with curved blades, likely for ventilation. The other is a shorter, gray cylindrical vent, possibly for plumbing or other utility exhaust. 


A wooden utility pole stands partially visible behind the roof. Several wires are strung from the pole, crossing the image horizontally. The wires are attached to the pole with gray insulators.


Behind the pole and wires are the branches and leaves of several trees, likely eucalyptus, given their appearance and the visible peeling bark. The trees appear to be quite tall and full. The sky above is a clear, bright blue.
-----

Following the Evaluation Steps:

**STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and fluency according to the criteria.**

* **Instruction following:** The response follows the instruction to describe the image. It provides a detailed description of the various elements present in the image.
* **Groundedness:** The response appears to be grounded in the image provided, describing only what is visible.  I cannot confirm 100% since I don't have access to the image. However, the description does not hallucinate any details.
* **Conciseness:** The response is slightly verbose. While it provides good detail, it could be more concise by combining some sentences. For example, the two vent descriptions could be combined into a single sentence.
* **Fluency:** The response is well-organized and easy to read. The language used is clear and descriptive, and the structure flows logically from one element of the image to the next.

**STEP 2: Score based on the rubric.**

The response scores a **4 (Good)**. It follows instructions, is grounded, and fluent. While it provides a comprehensive description, it could be slightly more concise.

-----

SummaryRating.GOOD


## 2. Few-shot Prompting

Let's modify the image understanding from a text prompt to a Few-Shot Prompt outputting JSON.

Capstone objectives covered:
* Structured prompting
* Few-shot prompting 
* Function calling

In [25]:
# Assign a few-shot prompt with examples.

base_prompt = "Using the following image, identify the image dimensions and mime type from file contents and catalog every object creating valid JSON:"

few_shot_examples = """Using the following image, identify the image dimensions and mime type from file contents and catalog every object creating valid JSON:

EXAMPLE:
For an 20 x 40 pixel (width x height) jpg image that has a basketball, a hand strainer, and a jade plant.

JSON Response:
```
{
"size": "20Wx40H",
"mime-type": "image/jpeg",
"contents": 
[
{   
"object": "ball", 
"type": "basketball", 
"colors": "orange, black",
"description": "an orange rubber basketball with faded black lines" 
},
{   
"object": "strainer", 
"type": "kitchen handheld strainer", 
"colors": "silver, blue",
"description": "a small hand strainer for washing small fruit and vegetables" 
},
{   
"object": "plant", 
"type": "jade plant", 
"colors": "green, orange",
"description": "a large, healthy jade plant with medium size leaves in an orange plastic pot" 
}
]
}
```

EXAMPLE:
For an 880 x 440 pixel (width x height) gif image that has a dog with a bone, a swimming pool, and a table with an parasol and a margarita.

JSON Response:
```
{
"size": "880Wx440H",
"mime-type": "image/gif",
"contents": 
[
{   
"object": "dog", 
"type": "labradoodle", 
"colors": "white, black",
"description": "a small black and white labradoodle lying on the ground chewing a bone" 
},
{   
"object": "bone", 
"type": "dog bone", 
"colors": "yellow",
"description": "a small yellow dog bone chew toy" 
},
{   
"object": "pool", 
"type": "swimming pool", 
"colors": "brown, blue, white",
"description": "a large brown round above ground swimming pool with white inner lining and blue water" 
},
{
"object": "table", 
"type": "patio table", 
"colors": "green, clear",
"description": "a round green patio table with clear glass top that has a large parasol resting in a hole in the middle and a full margarita tumbler" 
},
{
"object": "umbrella", 
"type": "parasol", 
"colors": "green",
"description": "a large open parasol creating shade behind the table" 
},
{
"object": "drink", 
"type": "margarita", 
"colors": "blue, yellow",
"description": "a large blue goblet containing a yellow liquid and ice" 
}
]
}
``````

Object Catalog:
"""

img_few_shot_prompt = base_prompt + few_shot_examples

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        response_mime_type="application/json"
    ),
    contents=[img_few_shot_prompt, img])

print(response.text)

{
"size": "1280Wx960H",
"mime-type": "image/jpeg",
"contents": 
[
{   
"object": "turbine vent", 
"type": "roof turbine vent", 
"colors": "silver",
"description": "a silver metal roof turbine vent" 
},
{   
"object": "vent pipe", 
"type": "roof vent pipe", 
"colors": "silver",
"description": "a silver metal roof vent pipe" 
},
{   
"object": "roof", 
"type": "shingle roof", 
"colors": "brown",
"description": "a brown shingle roof" 
},
{   
"object": "tree", 
"type": "eucalyptus tree", 
"colors": "green, brown",
"description": "a large eucalyptus tree with green leaves and brown branches" 
},
{   
"object": "pole", 
"type": "utility pole", 
"colors": "gray",
"description": "a gray wooden utility pole with wires" 
}
]
}


### Use extensions to fix hallucinations 

The JSON produced above is generally good, however the model can't figure out a way to extract metadata from the file without help.

So, let's give it the help! 
* Dimensions: The PIL library extracts image dimensions as *img.size*.
* MIME Type: The library mimetypes can take a swag. We can default it to "image/*file-extension*". 

In [35]:
import mimetypes

def get_img_mimetype () -> str:
  """Retrieve the mimetype of an image file."""
  image_file = img
  print(f' - FUNCTION CALL: get_img_mimetype({image_file.filename})')

  # The following line assumes that the image path is the current directory.
  img_mime_type, img_encoding = mimetypes.guess_type(image_file.filename)
  if img_mime_type == "None":
    img_mime_type = "image/" + image_file.format

  return img_mime_type

def get_img_size () -> str:
  """Retrieve the height and width of an image file in pixels."""
  image_file = img
  print(f' - FUNCTION CALL: get_img_size({image_file.filename})')

  return image_file.size

print(get_img_size())
print(get_img_mimetype())


 - FUNCTION CALL: get_img_size(IMG_1125.jpg)
(4032, 3024)
 - FUNCTION CALL: get_img_mimetype(IMG_1125.jpg)
image/jpeg


Change the prompt and API call to use an interactive chat. 

In [50]:
file_tools = [get_img_size, get_img_mimetype]

base_instruction = """You are an image analyzing chatbot that can interact with image file tools. You will accept user requests 
containing images, use image tools to identify necessary file attributes, and return output from tool functions and then a JSON response containing file attributes and 
a catalog of the elements.

Use the tool get_img_size to provide a list containing the W (width) and H (height) of the image in order, and use the tool get_img_mimetype 
to provide the image mimetype."""

instruction = base_instruction + few_shot_examples

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=file_tools
    ),
)

In [51]:
resp = chat.send_message(img)
print(f"\n{resp.text}")

 - FUNCTION CALL: get_img_size(IMG_1125.jpg)
 - FUNCTION CALL: get_img_mimetype(IMG_1125.jpg)

```json
{
"size": "4032Wx3024H",
"mime-type": "image/jpeg",
"contents": 
[
{   
"object": "turbine vent", 
"type": "roof turbine vent", 
"colors": "silver",
"description": "a silver metal roof turbine vent" 
},
{   
"object": "plumbing vent", 
"type": "roof plumbing vent", 
"colors": "grey",
"description": "a short grey roof plumbing vent" 
},
{   
"object": "shingles", 
"type": "roof shingles", 
"colors": "brown",
"description": "brown asphalt roof shingles" 
},
{
"object": "tree", 
"type": "large tree", 
"colors": "green, brown",
"description": "a very large tree with green leaves and brown branches" 
},
{
"object": "pole", 
"type": "utility pole", 
"colors": "grey",
"description": "a grey wooden utility pole with power lines attached" 
}
]
}
```


Ask more questions.

In [ ]:

resp = chat.send_message("Can you identify the manufacturer of anything on the roof? What items are you considering?")
print(f"\n{resp.text}")


I am unable to identify the manufacturer of any of the objects in the image using the available tools. I considered the turbine vent, plumbing vent, and shingles, but I am unable to determine their manufacturers.



Send other images.

In [60]:
img.close()
alt_images = ['IMG_1069.jpg', 'IMG_1106.jpg']
img = PIL.Image.open(alt_images[1])

resp = chat.send_message(img)
print(f"\n{resp.text}")

 - FUNCTION CALL: get_img_size(IMG_1106.jpg)
 - FUNCTION CALL: get_img_mimetype(IMG_1106.jpg)

```json
{
"size": "4032Wx3024H",
"mime-type": "image/jpeg",
"contents": 
[
{   
"object": "stop sign", 
"type": "stop sign", 
"colors": "red, white",
"description": "a red and white stop sign at a street corner" 
},
{   
"object": "street sign", 
"type": "street sign", 
"colors": "green, white",
"description": "a green and white street sign reading 'Trestle Glen'" 
},
{   
"object": "fire hydrant", 
"type": "fire hydrant", 
"colors": "silver",
"description": "a silver fire hydrant on the corner" 
},
{
"object": "parking meter", 
"type": "parking meter", 
"colors": "grey",
"description": "a grey parking meter on the sidewalk" 
},
{
"object": "cars", 
"type": "cars", 
"colors": "various",
"description": "several cars parked and driving on the street" 
},
{
"object": "tree", 
"type": "trees", 
"colors": "green",
"description": "several green trees" 
},
{
"object": "building", 
"type": "building"

In [61]:
resp = chat.send_message("List the cars in the image.")
print(f"\n{resp.text}")


Okay, here's a list of the cars I can identify in the image:

*   A grey sedan in the lower left corner
*   Several cars driving on the street in the background, including a dark sedan and a light-colored car
*   A silver SUV parked on the right side of the street
*   A black Tesla parked near the SUV.


## Postscript
This exercise was my dipping my first toe in the water of modern Generative AI. Both the strengh of the technology and the challenges seem to **click** with my historical knowledge about AI and optimization projects at Tepper. I'm eager to do more.

### Next Steps
My next project was a private one, where I used python to create tools to clean out an overstuffed old mailbox in MBOX format. 

I went through the gyrations of trying small local models using **Ollama**:
- qwen2.5-coder:latest
- llama3.1:8b-instruct-q8_0
- llama3.1:70b

I wrote the tools code as I would have for *Gemini*, and rewrote for *Qwen* and rewrote for *Llama3.1* before recognizing that the smaller models didn't provide enough context to execute a combination of the natural language interaction and data processing.

Waiting for llama3.1:70b I resolved to rewrite for Gemini, where it worked without a hitch. I realize now it's a better vibe coding exercise than something a small local model can assist me with. 